In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# GMM Class with Neccessary Supporting Functions

In [2]:
def GaussianProb(X, mean, cov_matrix):
    epsilon = 1e-4  # Small positive constant
    
    if np.linalg.det(cov_matrix) < 1e-4:
        cov_matrix = np.diag(epsilon+np.diag(cov_matrix))
    diff = X - mean  # Compute the differences for all rows at once
    exponent = -0.5 * np.sum(diff @ np.linalg.inv(cov_matrix) * diff, axis=1)
    
    prefactor = 1.0 / (np.sqrt(((2 * np.pi) ** X.shape[1]) * np.linalg.det(cov_matrix)))
    likelihoods = prefactor * np.exp(exponent)
    
    return likelihoods

In [3]:
from sklearn.cluster import KMeans

class GMM():
    def __init__(self , max_iter = 10 , initialisation = 'standard' , preprocess = False):
        self.k = 1
        self.num_cols = 1
        self.max_iter = max_iter
        self.means = None
        self.stds = None
        self.probabilities = None
        self.likelihoods = None
        self.loglikelihood = []
        self.initialisation = initialisation
        self.preprocess = preprocess

    def preprocesing(self , X):
        min_vals = np.min(X, axis=0)
        max_vals = np.max(X, axis=0)
        normalized_X = (X - min_vals) / (max_vals - min_vals)
        return normalized_X

    def initialise_parameters(self , X):
        self.num_cols = X.shape[1]
        
        kmeans = KMeans(n_clusters=self.k, init='k-means++' , n_init=10)  
        kmeans.fit(X)
        self.means = kmeans.cluster_centers_
        initial_covariances = []
        
        for cluster_label in range(self.k):
            cluster_points = X[kmeans.labels_ == cluster_label]
            cluster_covariance = np.cov(cluster_points, rowvar=False)
            initial_covariances.append(cluster_covariance)
        
        self.stds = np.array(initial_covariances)
        self.stds = np.array([np.diag(np.diag(std)) for std in self.stds])
        
        self.probabilities = np.ones(self.k)
        self.probabilities /= np.sum(self.probabilities)
       
    
    def random_initialisation(self , X):
        self.num_cols = X.shape[1]
        self.means = np.random.rand(self.k , self.num_cols)
        self.stds = np.random.rand(self.k , self.num_cols , self.num_cols)
        for i in range(self.k):
            self.stds[i] = np.diag(np.diag(self.stds[i]))
        self.probabilities = np.ones(self.k)
        self.probabilities /= np.sum(self.probabilities)
    
    def get_membership(self , X):
        r = np.zeros((X.shape[0] , self.k))
        for j in range(self.k):
            gaussian_probs_j = GaussianProb(X , self.means[j] , self.stds[j])
            r[: , j] = self.probabilities[j]*gaussian_probs_j
        
        r = r / np.sum(r , axis = 1 , keepdims = True)
        return r

    def get_data_likelihoods(self , X):
        return np.sum(self.likelihoods , axis = 0)

    def compute_likelihoods(self , X):
        likelihoods = np.zeros((self.k , X.shape[0]))
        for i in range(self.k):
            gaussian_probs = GaussianProb(X , self.means[i] , self.stds[i])
            likelihoods[i] = self.probabilities[i]*gaussian_probs

        likelihoods = likelihoods / np.sum(likelihoods , axis = 0)

        self.likelihoods = likelihoods

    
    def update_parameters(self , X):
        means =  np.zeros((self.k , self.num_cols))
        stds = np.zeros((self.k , self.num_cols , self.num_cols))
        probs = np.zeros(self.k)
      
        for i in range(self.k):
            probs[i] = np.sum(self.likelihoods[i])/X.shape[0]
            X_centered = X - self.means[i]
            for j in range(X.shape[0]):
                means[i] += self.likelihoods[i][j]*X[j]
                stds[i] += self.likelihoods[i][j]*(X_centered[j].T.dot(X_centered[j]))
            means[i] = means[i]/np.sum(self.likelihoods[i])
            stds[i] = stds[i]/np.sum(self.likelihoods[i])
        
        self.means = means 
        self.stds = stds
        
        self.probabilities = probs
    
    def get_loglikelihood(self , X):
        loglikelihood = np.zeros(X.shape[0])
        for i in range(self.k):
            gaussian_probs = GaussianProb(X , self.means[i] , self.stds[i]) # 2000 pts
            loglikelihood += self.probabilities[i]*gaussian_probs # 2000 pts
        # print(loglikelihood.shape)
        loglikelihood = np.sum(np.log(loglikelihood))
        return loglikelihood

    def get_aic(self , X):
        loglikelihood = self.get_loglikelihood(X)
        num_parameters = self.k*(2*self.num_cols + 1)
        aic = -2*loglikelihood + 2*num_parameters
        return aic

    def get_bic(self,X):
        N = X.shape[0]
        loglikelihood = self.get_loglikelihood(X)
        num_parameters = self.k*(2*self.num_cols + 1)
        bic = -2*loglikelihood + num_parameters*np.log(N)
        return bic
            
    def get_parameters(self):
        return self.means , self.stds , self.probabilities
    
    def fit(self , X , k):
        if self.preprocess:
            X = self.preprocesing(X)

        self.k = k
        
        if self.initialisation == 'random':
            self.random_initialisation(X)
        else:
            self.initialise_parameters(X)
        
        for i in range(self.max_iter):
            self.compute_likelihoods(X)
            self.update_parameters(X)
            self.loglikelihood.append(self.get_loglikelihood(X))
            
    
    def predict(self , X):
        probs = np.zeros((X.shape[0] , self.k))
        for i in range(self.k):
            probs[:,i] = self.probabilities[i]*GaussianProb(X , self.means[i] , self.stds[i])
        return np.argmax(probs , axis = 1)
        

# Customer's Dataset

In [18]:

data = pd.read_csv('../Data/SMAI-Dataset-customer-dataset/data.csv')
data.head()
X_data = np.array(data.iloc[:, 1:])

In [19]:
model = GMM(max_iter = 10 , initialisation='random' , preprocess=True)
model.fit(X_data , 3)
means , covariances , probs = model.get_parameters()

In [20]:
import plotly.graph_objs as go
import pandas as pd

k = 3  # Number of Gaussian components
n_dimensions = 7  # Dimensionality of the data


# Create a DataFrame for the GMM parameters
gmm_df = pd.DataFrame({
    'Component': [f'Component {i + 1}' for i in range(k)],
    'Mean': [str(mean) for mean in means],
    'Covariance Matrix': [str(cov) for cov in covariances],
    'Probability': probs
})

# Create a table using Plotly
table_trace = go.Table(
    header=dict(values=['Component', 'Mean', 'Covariance Matrix', 'Probability']),
    cells=dict(values=[gmm_df['Component'], gmm_df['Mean'], gmm_df['Covariance Matrix'], gmm_df['Probability']])
)

# Create a layout for the table
layout = go.Layout(
    title='Gaussian Mixture Model Parameters',
)

# Create the figure and plot the table
fig = go.Figure(data=[table_trace], layout=layout)
fig.show()


In [21]:
ll = model.loglikelihood
xs = np.arange(1 , 11)
fig = px.line(x = xs , y = ll, title='Log Likelihood vs Iterations')
fig.show()

# Wine Dataset

In [25]:
from sklearn.datasets import load_wine
from sklearn.mixture import GaussianMixture

data = load_wine()
X_wine = data.data
y_wine = data.target



In [26]:
# plot the data
fig = px.scatter_3d(x=X_wine[:,0], y=X_wine[:,1], z = X_wine[:,2] ,color=y_wine)
fig.show()

## FINDING OPTIMAL NUMBER OF CLUSTERS FOR GMM

In [27]:
import plotly.graph_objs as go

n_clusters = np.arange(1 , 10 , 1)
models = [GMM(max_iter = 50) for i in n_clusters]
for i in range(len(models)):
    models[i].fit(X_wine, n_clusters[i])
aic = [models[i].get_aic(X_wine) for i in range(1 , len(models))]
bic = [models[i].get_bic(X_wine) for i in range(1 ,len(models))]



# Data for the first line
trace1 = go.Scatter(
    x=n_clusters,
    y=aic,
    mode='lines',
    name='AIC'
)

# Data for the second line
trace2 = go.Scatter(
    x=n_clusters,
    y=bic,
    mode='lines',
    name='BIC'
)

# Create a layout
layout = go.Layout(
    title='AIC/BIC vs Number of Clusters',
    xaxis=dict(title='X-Axis'),
    yaxis=dict(title='Y-Axis')
)

# Create a figure that combines both traces
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Show the plot
fig.show()


The ideal number of clusters are thus 2

## PERFORMING PCA TO APPLY CLUSTERING

In [29]:

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_wine)
y_wine = data.target

my_gmm = GMM(max_iter = 50)
kmeans_model = KMeans(n_clusters=3, random_state=42 , n_init=10)

my_gmm.fit(X_pca , 3)
kmeans_model.fit(X_pca)

my_gmm_labels = my_gmm.predict(X_pca)
kmeans_labels = kmeans_model.predict(X_pca)


fig = px.scatter(x=X_pca[:,0], y=X_pca[:,1], color=y_wine , title = 'Ground Truth')
fig.show()

fig = px.scatter(x=X_pca[:,0], y=X_pca[:,1], color=my_gmm_labels , title = 'My GMM')
fig.show()

fig = px.scatter(x=X_pca[:,0], y=X_pca[:,1], color=kmeans_labels , title = 'KMeans')
fig.show()


## SILHOUTTE SCORE COMPARISON

In [12]:
from sklearn.metrics import silhouette_score
my_gmm_score = silhouette_score(X_pca , my_gmm_labels)
kmeans_score = silhouette_score(X_pca , kmeans_labels)

scores = [my_gmm_score , kmeans_score]
labels = ['My GMM' , 'KMeans']

fig = px.bar(x=labels, y=scores , title = 'Silhouette Scores')
fig.show()